# Imports

In [ ]:
import sys
sys.path.append('/home/jovyan/work')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Importar utilitários customizados
from src import S3Client, DatabaseClient

# Configurações de visualização
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

import warnings
warnings.filterwarnings('ignore')

# Carregamento dos Dados

## Conectar ao MinIO

In [ ]:
# Conectar ao MinIO
s3 = S3Client()

In [ ]:
# Listar arquivos disponíveis
print("📋 Arquivos no MinIO:")
files = s3.list_files()
for f in files:
    print(f"   - {f}")

# %% Carregar dataset bruto
df = s3.read_csv('raw/heart.csv')

print(f"\n✅ Dataset carregado!")
print(f"📊 Shape: {df.shape[0]} linhas, {df.shape[1]} colunas")

# Visão Geral dos Dados

## Informações Básicas

In [ ]:
print("\n--- Primeiras 5 Linhas: ---")
display(df.head())

In [ ]:
print("\n--- Informações do DataFrame: ---")
df.info()

In [ ]:
print(f"\n📊 O DataFrame tem {df.shape[0]} linhas e {df.shape[1]} colunas.")
print(f"\n📋 Colunas: {list(df.columns)}")

In [ ]:
print("\n--- Resumo Estatístico: ---")
display(df.describe())

In [ ]:
print("\n--- Tipos de Dados: ---")
print(df.dtypes)

## Análise da Qualidade dos Dados

### Valores Nulos

In [ ]:
print("\n--- Avaliação dos Valores Nulos: ---")
null_counts = df.isnull().sum()
display(null_counts[null_counts > 0] if null_counts.sum() > 0 else "✅ Nenhum valor nulo encontrado!")

### Duplicatas

In [ ]:
print("\n--- Avaliação dos Registros Duplicados: ---")
n_duplicates = df.duplicated().sum()
print(f"Total de linhas duplicadas: {n_duplicates} ({n_duplicates/len(df)*100:.2f}%)")

if n_duplicates > 0:
    print("\n⚠️ ATENÇÃO: Dataset tem 272 duplicatas (23% dos dados)!")
    print("Serão removidas no pré-processamento.")

### Valores Impossíves

In [ ]:
def detectar_valores_impossiveis(df):
    """
    Ranges fisiológicos baseados em guidelines médicas (AHA, NIH, ACC)
    """
    ranges = {
        "age": (18, 100),
        "resting bp s": (60, 260),
        "cholesterol": (70, 900),
        "max heart rate": (30, 260),
        "oldpeak": (-3.0, 7.0)
    }
    
    resultados = {}
    
    for col, (lo, hi) in ranges.items():
        impossiveis = df[(df[col] < lo) | (df[col] > hi)]
        resultados[col] = {
            "total_impossiveis": len(impossiveis),
            "percentual": len(impossiveis) / len(df) * 100,
            "indices": impossiveis.index.tolist()[:10],  # Primeiros 10
            "valores": impossiveis[col].tolist()[:10]
        }
    
    return resultados

In [ ]:
print("\n--- Detecção de Valores Impossíveis: ---")
resultados = detectar_valores_impossiveis(df)

for col, info in resultados.items():
    print(f"\n==== {col} ====")
    print(f"Total impossíveis: {info['total_impossiveis']} ({info['percentual']:.2f}%)")
    if info['total_impossiveis'] > 0:
        print(f"Exemplos: {info['valores']}")

print("\n⚠️ PRINCIPAIS PROBLEMAS IDENTIFICADOS:")
print("   1. resting bp s = 0 (1 registro)")
print("   2. cholesterol = 0 (172 registros = 14.45%!)")
print("   → Serão substituídos pela mediana no pré-processamento")

### Outliers

In [ ]:
def detect_outliers_iqr(df, column):
    """Detecta outliers usando método IQR"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()

In [ ]:
print("\n--- Avaliação dos Outliers (IQR): ---")
numeric_features = df.select_dtypes(include=[np.number]).columns.drop('target')

outlier_summary = {}
for col in numeric_features:
    n_outliers = detect_outliers_iqr(df, col)
    if n_outliers > 0:
        outlier_summary[col] = n_outliers

if outlier_summary:
    print("Outliers detectados por feature:")
    for col, count in outlier_summary.items():
        print(f"  - {col}: {count} outliers")
else:
    print("Nenhum outlier extremo detectado")

print("\n💡 Nota: Outliers serão mantidos pois são fisiologicamente possíveis")

# Visualizações

## Outliers por Target

In [ ]:
TARGET_COLUMN = 'target'
numeric_cols = [c for c in df.select_dtypes(include='number').columns if c != TARGET_COLUMN]

n_cols = 3 
n_rows = int(np.ceil(len(numeric_cols) / n_cols))

plt.figure(figsize=(n_cols * 6, n_rows * 5))
plt.suptitle("Detecção de Outliers (IQR) nas Colunas Numéricas", fontsize=18)

for i, col in enumerate(numeric_cols, 1):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers_df = df[(df[col] < lower) | (df[col] > upper)]
    
    plt.subplot(n_rows, n_cols, i)
    
    sns.stripplot(
        x=TARGET_COLUMN, 
        y=col, 
        data=df, 
        hue=TARGET_COLUMN, 
        palette="Set1", 
        jitter=True,
        legend=False
    )
    
    plt.axhline(upper, color="red", linestyle="--", label="IQR Upper")
    plt.axhline(lower, color="red", linestyle="--", label="IQR Lower")
    
    plt.title(f"{col} (outliers: {len(outliers_df)})")
    plt.xlabel("Target")
    plt.ylabel(col)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

## Distribuição da Variável Alvo

In [ ]:
print("\n--- Distribuição das Classes do Target ---")
target_counts = df["target"].value_counts()
target_percent = df["target"].value_counts(normalize=True) * 100

target_df = pd.DataFrame({
    "Contagem": target_counts,
    "Percentual (%)": target_percent.round(2)
})

display(target_df)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Contagem
axes[0].bar(target_counts.index, target_counts.values, color=['#3498db', '#e74c3c'])
axes[0].set_xlabel('Target')
axes[0].set_ylabel('Contagem')
axes[0].set_title('Distribuição do Target (Contagem)')
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['Saudável (0)', 'Doença (1)'])

# Percentual
axes[1].pie(target_counts.values, labels=['Saudável (0)', 'Doença (1)'], 
            autopct='%1.1f%%', colors=['#3498db', '#e74c3c'], startangle=90)
axes[1].set_title('Distribuição do Target (Percentual)')

plt.tight_layout()
plt.show()

## Análise Univariada - Distribuições

### Distribuições das features numéricas

In [ ]:
numerical_cols = ["age", "resting bp s", "cholesterol", 
                  "fasting blood sugar", "max heart rate", "oldpeak"]

plt.figure(figsize=(16, 10))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True, color='steelblue')
    plt.title(f"Distribuição de {col}")
    plt.xlabel(col)
    plt.ylabel("Frequência")
plt.tight_layout()
plt.show()

### Features binárias

In [ ]:
binary_cols = ["sex", "exercise angina"]

plt.figure(figsize=(12, 4))
for i, col in enumerate(binary_cols, 1):
    plt.subplot(1, 2, i)
    df[col].value_counts().plot(kind='bar', color=['#3498db', '#e74c3c'])
    plt.title(f"Distribuição de {col}")
    plt.xlabel(col)
    plt.ylabel("Contagem")
    plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Features categóricas

In [ ]:
categorical_cols = ["chest pain type", "resting ecg", "ST slope"]

plt.figure(figsize=(15, 4))
for i, col in enumerate(categorical_cols, 1):
    plt.subplot(1, 3, i)
    df[col].value_counts().sort_index().plot(kind='bar', color='steelblue')
    plt.title(f"Distribuição de {col}")
    plt.xlabel(col)
    plt.ylabel("Contagem")
    plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## Análise de Correlação

### Matriz de correlação completa

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(df.corr(), cmap="coolwarm", annot=False, cbar_kws={'label': 'Correlação'})
plt.title("Matriz de Correlação - Todas as Features")
plt.tight_layout()
plt.show()

### Correlação entre features numéricas

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df[numerical_cols].corr(), cmap="coolwarm", annot=True, fmt='.2f')
plt.title("Correlação entre Variáveis Numéricas")
plt.tight_layout()
plt.show()

### Correlação com o target

corr_with_target = df.corr()["target"].abs().sort_values(ascending=False)

plt.figure(figsize=(8, 10))
sns.barplot(x=corr_with_target.values, y=corr_with_target.index, palette='viridis')
plt.title("Correlação das Variáveis com o Target (valor absoluto)")
plt.xlabel("Correlação Absoluta")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

print("\n🔝 Top 5 features mais correlacionadas com target:")
display(corr_with_target.head(6))

## Análise Bivariada - Features vs Target

### Boxplots por target

In [ ]:
plt.figure(figsize=(16, 10))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x="target", y=col, data=df, palette="Set2")
    plt.title(f"{col} por Target")
    plt.xlabel("Target (0=Saudável, 1=Doença)")
    plt.ylabel(col)
plt.tight_layout()
plt.show()

### Violin plots

In [ ]:
plt.figure(figsize=(16, 10))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.violinplot(x="target", y=col, data=df, palette="muted")
    plt.title(f"{col} por Target")
    plt.xlabel("Target (0=Saudável, 1=Doença)")
    plt.ylabel(col)
plt.tight_layout()
plt.show()

### KDE plots

plt.figure(figsize=(16, 10))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.kdeplot(data=df, x=col, hue="target", fill=True, palette="Set1")
    plt.title(f"Densidade de {col} por Target")
    plt.xlabel(col)
    plt.ylabel("Densidade")
plt.tight_layout()
plt.show()

## Pairplot (Features Numéricas)

In [ ]:
print("\n⏳ Gerando pairplot (pode demorar)...")
sns.pairplot(df[numerical_cols + ["target"]], hue="target", palette="Set1", corner=True)
plt.suptitle("Pairplot - Features Numéricas vs Target", y=1.01)
plt.show()

📝 Resumo das Descobertas

⚠️ Problemas Identificados:
1. **Duplicatas**: 272 linhas (23% dos dados) → Remover
2. **Valores impossíveis**:
   - `resting bp s = 0`: 1 registro → Substituir por mediana
   - `cholesterol = 0`: 172 registros (14.45%) → Substituir por mediana
3. **Outliers**: Detectados mas fisiologicamente possíveis → Manter

✅ Características Positivas:
1. **Sem valores nulos**
2. **Classes balanceadas**: 55% vs 45%

# Salvar no PostgreSQL

In [ ]:
# Conectar ao PostgreSQL e salvar estatísticas básicas
db = DatabaseClient()

# Estatísticas básicas para documentação
stats = {
    'total_samples': len(df),
    'total_features': len(df.columns),
    'duplicates': n_duplicates,
    'impossible_values': sum([r['total_impossiveis'] for r in resultados.values()]),
    'class_0': int(target_counts[0]),
    'class_1': int(target_counts[1])
}

print("\n📊 Estatísticas do Dataset:")
for key, value in stats.items():
    print(f"   {key}: {value}")